In [1]:
pwd

'/private/home/dpf/projects/codex/notebooks'

In [2]:
cd /private/home/dpf/projects/codex

/private/home/dpf/projects/codex


In [11]:
from typewriter import *

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
pdb on

Automatic pdb calling has been turned ON


In [11]:
import ast
import astunparse

In [22]:
from type_hints import normalize_type

In [6]:
typewriter_dir = "/private/home/dpf/data/TypeWriter_dataset"
crawl_root = "/checkpoint/dpf/data/typewriter/crawl/data"
split = 'validation'


In [52]:
decontaminate = True
imports_and_function_only = True

examples = build_examples(typewriter_dir, crawl_root, imports_and_function_only, split=split, show_tqdm=True)

before removing crawled repos: 272
after removing crawled repos: 272
archive /checkpoint/dpf/data/typewriter/crawl/data/productml/blurr.json.zstd not found; removing from repos
archive /checkpoint/dpf/data/typewriter/crawl/data/b2wdigital/aiologger.json.zstd not found; removing from repos
archive /checkpoint/dpf/data/typewriter/crawl/data/CasperLabs/CasperLabs.json.zstd not found; removing from repos


100%|████████████████████████████████████████████████████████████████████████████| 16492/16492 [01:50<00:00, 149.51it/s]

Counter({'no repo': 11034, 'no ast match': 547, 'file mismatch 0': 69})
return: skipped 11650 / 16492 examples (4842 remaining)


In [35]:
(80341 - 7383) // 5

14591

In [55]:
typewriter_predictions = get_typewriter_predictions(read_prediction_file(typewriter_dir, argument=False))
evaluate(typewriter_predictions)

{'precision': 0.744053851907255,
 'recall': 0.6032015522677662,
 'f1': 0.6662648181635523,
 'n_correct': 9948,
 'n_predictions': 13370,
 'n_types': 16492}

In [54]:
typewriter_predictions_filtered = get_typewriter_predictions(examples)
evaluate(typewriter_predictions_filtered)

{'precision': 0.7539279671259367,
 'recall': 0.6441553077240809,
 'f1': 0.69473215280098,
 'n_correct': 3119,
 'n_predictions': 4137,
 'n_types': 4842}

In [60]:
typewriter_predictions_filtered = get_typewriter_predictions(examples)
evaluate(typewriter_predictions_filtered[:100], type_from_source=True)

{'precision': 0.5952380952380952,
 'recall': 0.5,
 'f1': 0.5434782608695652,
 'n_correct': 50,
 'n_predictions': 84,
 'n_types': 100}

In [ ]:
pdb on

In [7]:
import re
import json
import os

line_url_re = re.compile(r'https://github.com/(.*)/(.*)\.git::(.*)')
arg_re = re.compile(r'(\d+)->\[(.*)\]')

def read_github_url_file(path):
    repo_to_commit = {}
    repo_to_name = {}
    with open(path, 'r') as f:
        for line in f:
            match = line_url_re.match(line.strip())
            user, repo, commit = match.groups()
            assert repo not in repo_to_commit
            repo_to_commit[repo] = commit
            repo_to_name[repo] = f'{user}/{repo}'
    return repo_to_commit, repo_to_name

def read_filename_file(path):
    names_to_paths = {}
    with open(path, 'r') as f:
        for line in f:
            line = line.strip()
            split = line.split('/')
            repo, *rest = split
            name = repo
            if name not in names_to_paths:
                names_to_paths[name] = []
            names_to_paths[name].append('/'.join(rest))
    return names_to_paths

def read_prediction_file(path, argument=False):
    prefix = "/home/anon/local/github-repos/"
    with open(path, 'r') as f:
        data = json.load(f)
    records = []
    for key, answers in data.items():
        key = key.strip()
        file, location = key.split(' : ')
        if argument:
            line_num, var = arg_re.match(location).groups()
            line_num = int(line_num)
        else:
            line_num = int(location)
        assert file.startswith(prefix)
        repo, *rest = file.lstrip(prefix).split('/')
        path = '/'.join(rest)
        gold = answers[0]
        predicted = answers[1]
        record = {
            'repo': repo,
            'path': path,
            'line_number': line_num,
            'true_type': gold,
            'predicted_types': predicted
        }
        if argument:
            record['variable'] = var
        records.append(record)
    return records

def read_typewriter_validation(typewriter_dir):
    repo_to_commit, repo_to_name = read_github_url_file(os.path.join(typewriter_dir, 'github_urls.txt'))
    # Dict[str, List[str]]: name -> paths
    repo_to_paths = read_filename_file(os.path.join(typewriter_dir, 'open_source_validation_files.txt'))
    return repo_to_commit, repo_to_name, repo_to_paths

In [5]:
repo_to_commit, repo_to_name, repo_to_path = read_typewriter_validation('/private/home/dpf/data/TypeWriter_dataset/')

In [8]:
predictions = read_prediction_file('/private/home/dpf/data/TypeWriter_dataset/open_source_results_return.json')

In [9]:
arg_predictions = read_prediction_file('/private/home/dpf/data/TypeWriter_dataset/open_source_results_argument.json', argument=True)

In [10]:
len(arg_predictions)

21215

In [11]:
arg_predictions[0]

{'repo': 'ycharm-course',
 'path': 'extra_samples/parsing/numparser.py',
 'line_number': 4,
 'true_type': 'str',
 'predicted_types': ['str', 'bytes', 'Text', 'Optional[str]', 'LongTensor'],
 'variable': 'text'}

In [12]:
validation_names = set(
    repo_to_name[repo] for repo in repo_to_path
)

In [13]:
len(validation_names)

272

In [14]:
import csv
import glob

In [15]:
crawled_repos = set()
for fname in glob.glob('/checkpoint/dpf/data/github/*.csv'):
    with open(fname, 'r') as f:
        for record in csv.DictReader(f):
            if 'name' in record:
                key = 'name'
            else:
                key = 'repo_name'
                assert key in record
            crawled_repos.add(record[key])

In [16]:
len(crawled_repos)

681590

In [17]:
repo_to_path_decon = {
    repo: paths for repo, paths in repo_to_path.items()
    if repo_to_name[repo] not in crawled_repos
}

In [18]:
sum(len(paths) for repo, paths in repo_to_path.items())

1936

In [19]:
sum(len(paths) for repo, paths in repo_to_path_decon.items())

1229

In [20]:
1229 / 1936

0.6348140495867769

In [21]:
crawl_root = '/checkpoint/dpf/data/typewriter/crawl/data'

In [22]:
import zstandard as zstd
import json

In [23]:
compressor = zstd.ZstdDecompressor()

In [24]:
repo_to_archive_decon = {}
for repo, paths in repo_to_path_decon.items():
    name = repo_to_name[repo]
    archive_fname = os.path.join(crawl_root, f"{name}.json.zstd")
    if not os.path.exists(archive_fname):
        print(f"archive {archive_fname} not found")
        continue
    with open(archive_fname, 'rb') as f:
        reader = compressor.stream_reader(f)
        archive = json.load(reader)
        repo_to_archive_decon[repo] = archive

archive /checkpoint/dpf/data/typewriter/crawl/data/productml/blurr.json.zstd not found
archive /checkpoint/dpf/data/typewriter/crawl/data/CasperLabs/CasperLabs.json.zstd not found


In [25]:
#!pip install astunparse

In [26]:
predictions[0]

{'repo': 'ycharm-course',
 'path': 'extra_samples/parsing/numparser.py',
 'line_number': 4,
 'true_type': 'List[int]',
 'predicted_types': ['JavaObject', 'SpanContext', 'ndarray', 'dtype', 'Model']}

In [28]:
predictions_decon = [pred for pred in predictions if pred['repo'] in repo_to_archive_decon]

In [29]:
arg_predictions_decon = [pred for pred in arg_predictions if pred['repo'] in repo_to_archive_decon]

In [30]:
len(predictions)

16492

In [31]:
len(predictions_decon)

4680

In [32]:
len(arg_predictions_decon)

5121

In [33]:
len(arg_predictions)

21215

In [34]:
len(arg_predictions_decon) / len(arg_predictions)

0.24138581192552439

In [35]:
predictions_decon[0]

{'repo': 'dbt',
 'path': 'core/dbt/node_runners.py',
 'line_number': 74,
 'true_type': 'Any',
 'predicted_types': ['None', 'Dict[str, Any]', 'str', '@@UNK@@', 'List[str]']}

In [36]:
file_data = repo_to_archive_decon[predictions_decon[0]['repo']]['file_data']

In [37]:
datum = next(datum for datum in file_data if datum['path'] == predictions_decon[0]['path'])

In [39]:
%load_ext autoreload
%autoreload 2

In [40]:
content = datum['content']

In [41]:
import type_hints

In [44]:
import ast, astunparse
from type_hints import TypeHintRemover, derive_prefix_suffix, TypeHintKeepOnlyTargeted

In [49]:
processor.matches

[{'node': <_ast.FunctionDef at 0x7f9c56560990>,
  'returns': <_ast.Name at 0x7f9c56560b90>}]

In [63]:
archive

{'commit_info': [{'authored_date': 1582613019,
   'commit_id': 'a5c83dad8382a1f407a0158ff14514f9369b265e',
   'committed_date': 1582613019,
   'diffs': [{'diff': '', 'path': 'todo/views/import_csv.py'}]}],
 'commits_and_paths': [{'commit_id': 'a5c83dad8382a1f407a0158ff14514f9369b265e',
   'paths': ['todo/views/import_csv.py']}],
 'file_data': [{'commit_id': 'a5c83dad8382a1f407a0158ff14514f9369b265e',
   'content': 'from django.contrib import messages\nfrom django.contrib.auth.decorators import login_required, user_passes_test\nfrom django.http import HttpResponse\nfrom django.shortcuts import redirect, render, reverse\n\nfrom todo.operations.csv_importer import CSVImporter\nfrom todo.utils import staff_check\n\n\n@login_required\n@user_passes_test(staff_check)\ndef import_csv(request) -> HttpResponse:\n    """Import a specifically formatted CSV into stored tasks.\n    """\n\n    ctx = {"results": None}\n\n    if request.method == "POST":\n        filepath = request.FILES.get("csvfile")

In [59]:
def create_return_example(source: str, lineno: int, return_type: str, imports_and_function=True):
    def match_with_type(function, returns):
        return astunparse.unparse(returns).strip() == return_type
    def match_with_line_and_type(function, returns):
        return match_with_type(function, returns) and lineno == function.lineno
    processor = TypeHintKeepOnlyTargeted(['return'], match_with_line_and_type)
    parsed_source = ast.parse(content)
    # remove the type annotations, except for the target
    processor.visit(parsed_source)
    assert len(processor.matches) == 1
    # now reparse, but only needing to keep the type
#     processor = TypeHintKeepOnlyTargeted(['return'], match_with_type, remove_type_imports=False)
#     processor.visit(ast.parse(clean_source))
#     assert len(procsssor.matches) == 1
#     clean_lines = clean_source.splitlines()
    
    if imports_and_function:
        extra_left = [astunparse.unparse(node).strip() for node in processor.imports]
        to_split = astunparse.unparse(processor.matches[0]['node'])
    else:
        extra_left = []
        to_split = astunparse.unparse(parsed_source)
    pairs = list(derive_prefix_suffix(to_split, f" -> {return_type}"))
    assert len(pairs) == 1
    left, right = pairs[0]
    return {
        'extra_left': extra_left,
        'left': left + ' -> ',
        'right': right,
        'return_type': return_type,
    }

In [60]:
create_return_example(content, rpedi, 'Any')

{'extra_left': ['import abc',
  'import threading',
  'import time',
  'import traceback',
  'from typing import List, Dict, Any, Optional',
  'from dbt import deprecations',
  'from dbt.adapters.base import BaseRelation',
  'from dbt.clients.jinja import MacroGenerator',
  'from dbt.compilation import compile_node',
  'from dbt.context.providers import generate_runtime_model',
  'from dbt.contracts.graph.manifest import Manifest',
  'from dbt.contracts.results import RunModelResult, collect_timing_info, SourceFreshnessResult, PartialResult',
  'from dbt.exceptions import NotImplementedException, CompilationException, RuntimeException, InternalException, missing_materialization',
  'from dbt.logger import GLOBAL_LOGGER as logger',
  'from dbt.node_types import NodeType',
  'import dbt.exceptions',
  'import dbt.tracking',
  'import dbt.ui.printer',
  'import dbt.flags'],
 'left': '\n\n@abc.abstractmethod\ndef compile(self, manifest) -> ',
 'right': ':\n    pass\n',
 'return_type': 'Any

In [50]:
def match(function, returns):
    return function.lineno == 74 and astunparse.unparse(returns).strip() == 'Any'
print(astunparse.unparse(parsed_source))


import abc
import threading
import time
import traceback
from typing import List, Dict, Any, Optional
from dbt import deprecations
from dbt.adapters.base import BaseRelation
from dbt.clients.jinja import MacroGenerator
from dbt.compilation import compile_node
from dbt.context.providers import generate_runtime_model
from dbt.contracts.graph.manifest import Manifest
from dbt.contracts.results import RunModelResult, collect_timing_info, SourceFreshnessResult, PartialResult
from dbt.exceptions import NotImplementedException, CompilationException, RuntimeException, InternalException, missing_materialization
from dbt.logger import GLOBAL_LOGGER as logger
from dbt.node_types import NodeType
import dbt.exceptions
import dbt.tracking
import dbt.ui.printer
import dbt.flags
INTERNAL_ERROR_STRING = 'This is an error in dbt. Please try again. If the error persists, open an issue at https://github.com/fishtown-analytics/dbt\n'.strip()

def track_model_run(index, num_nodes, run_model_result):
    if

In [43]:
pdb on

Automatic pdb calling has been turned ON


In [60]:
source = content

In [61]:
import pprint

In [64]:
parsed_source = ast.parse(source)
clean_ast = TypeHintRemover(len(source), preserve_other_types=False, remove_type_imports=False).visit(parsed_source)

In [69]:
print(astunparse.unparse(clean_ast))


import abc
import threading
import time
import traceback
from typing import List, Dict, Any, Optional
from dbt import deprecations
from dbt.adapters.base import BaseRelation
from dbt.clients.jinja import MacroGenerator
from dbt.compilation import compile_node
from dbt.context.providers import generate_runtime_model
from dbt.contracts.graph.manifest import Manifest
from dbt.contracts.results import RunModelResult, collect_timing_info, SourceFreshnessResult, PartialResult
from dbt.exceptions import NotImplementedException, CompilationException, RuntimeException, InternalException, missing_materialization
from dbt.logger import GLOBAL_LOGGER as logger
from dbt.node_types import NodeType
import dbt.exceptions
import dbt.tracking
import dbt.ui.printer
import dbt.flags
INTERNAL_ERROR_STRING = 'This is an error in dbt. Please try again. If the error persists, open an issue at https://github.com/fishtown-analytics/dbt\n'.strip()

def track_model_run(index, num_nodes, run_model_result):
    if

In [67]:
astunparse.unparse(clean_ast).splitlines()[72:76]

['    def _build_run_result(self, node, start_time, error, status, timing_info, skip=False, fail=None, warn=None, agate_table=None):',
 '        execution_time = (time.time() - start_time)',
 '        thread_id = threading.current_thread().name',
 '        return RunModelResult(node=node, error=error, skip=skip, status=status, fail=fail, warn=warn, execution_time=execution_time, thread_id=thread_id, timing=timing_info, agate_table=agate_table)']

In [62]:
parsed_source = ast.parse(source)
clean_source = astunparse.unparse(TypeHintRemover(len(source), preserve_other_types=False, remove_type_imports=False).visit(ast.parse(source)))
unique = set()
for i in range(len(source)):
    parsed_source = ast.parse(source)
    transformer = TypeHintRemover(i, preserve_other_types=False, remove_type_imports=False)
    transformed_ast = transformer.visit(parsed_source)
    transformed = astunparse.unparse(transformed_ast)
    if transformer.guard_value is not None:
        unique.add((transformed, transformed_ast, transformer.guard_value, transformer.guard_node))
infills = []
for (removed_source, removed_ast, removed_value, removed_node) in unique:
    if removed_value is None:
        continue
    for left, right in derive_prefix_suffix(clean_source, removed_value.strip()):
#         print("left:", left)
#         print("right:", right)
#         print("removed:", removed_value)
        infills.append((left,right,removed_value))
# pprint.pprint(infills)

KeyboardInterrupt: 

> /private/home/dpf/.conda/envs/codex/lib/python3.7/ast.py(35)parse()
     33     Equivalent to compile(source, filename, mode, PyCF_ONLY_AST).
     34     """
---> 35     return compile(source, filename, mode, PyCF_ONLY_AST)
     36 
     37 

ipdb> q


In [130]:
# for transformed, transformed_ast, value, node in unique:
#     if node.lineno == 20 and value.strip() == 'PlayerID':
#         print("found it")
#         break

found it


In [59]:
# content = '''
# """Type used for pivot coll for basic game model training"""
# from default_imports import *

# from modules.game.Game import GameID, PlayerID

# from pymongo.collection import Collection

# BasicGameActivationID = NewType('BasicGameActivationID', str)
# Prediction = NewType('Prediction', int)

# class BasicGameActivation(NamedTuple('BasicGameActivation', [
#         ('id', BasicGameActivationID),
#         ('gameId', GameID),
#         ('playerId', PlayerID),
#         ('engine', bool),
#         ('prediction', int)
#     ])):
#     @staticmethod
#     def fromPrediction(gameId: GameID, playerId: PlayerID, prediction: Prediction, engine: bool):
#         return BasicGameActivation(
#             id = gameId + '/' + playerId,
#             gameId = gameId,
#             playerId = playerId,
#             engine = engine,
#             prediction = prediction
#             )

#     @staticmethod
#     def makeId(gameId: GameID, playerId: PlayerID) -> BasicGameActivationID:
#         return gameId + '/' + playerId

# class BasicGameActivationBSONHandler:
#     @staticmethod
#     def reads(bson: Dict) -> BasicGameActivation:
#         return BasicGameActivation(
#             id = bson['_id'],
#             gameId = bson['gameId'],
#             playerId = bson['userId'],
#             engine = bson['engine'],
#             prediction = bson['prediction'])

#     @staticmethod
#     def writes(gba: BasicGameActivation) -> Dict:
#         return {
#             '_id': gba.id,
#             'gameId': gba.gameId,
#             'userId': gba.playerId,
#             'engine': gba.engine,
#             'prediction': gba.prediction
#         }

# class BasicGameActivationDB(NamedTuple('BasicGameActivationDB', [
#         ('basicGameActivationColl', Collection)
#     ])):
#     def byPlayerId(self, playerId: PlayerID) -> List[BasicGameActivation]:
#         return [BasicGameActivationBSONHandler.reads(bson) for bson in self.basicGameActivationColl.find({'userId': playerId})]

#     def byEngineAndPrediction(self, engine: bool, prediction: Prediction, limit: Opt[int] = None) -> List[BasicGameActivation]:
#         gtlt = '$gte' if engine else '$lte'
#         pipeline = [{'$match': {'engine': engine, 'prediction': {gtlt: prediction}}}]

#         if limit is not None:
#             pipeline.append({'$sample': {'size': limit}})

#         return [BasicGameActivationBSONHandler.reads(bson) for bson in self.basicGameActivationColl.aggregate(pipeline)]

#     def write(self, gba: BasicGameActivation):
#         self.basicGameActivationColl.update_one({'_id': gba.id}, {'$set': BasicGameActivationBSONHandler.writes(gba)}, upsert=True)

#     def writeMany(self, gbas: List[BasicGameActivation]):
#         [self.write(gba) for gba in gbas]
# '''

In [131]:
node.lineno

20

In [133]:
node.col_offset

49

In [135]:
print(transformed)


'Type used for pivot coll for basic game model training'
from default_imports import *
from modules.game.Game import GameID, PlayerID
from pymongo.collection import Collection
BasicGameActivationID = NewType('BasicGameActivationID', str)
Prediction = NewType('Prediction', int)

class BasicGameActivation(NamedTuple('BasicGameActivation', [('id', BasicGameActivationID), ('gameId', GameID), ('playerId', PlayerID), ('engine', bool), ('prediction', int)])):

    @staticmethod
    def fromPrediction(gameId, playerId, prediction, engine):
        return BasicGameActivation(id=((gameId + '/') + playerId), gameId=gameId, playerId=playerId, engine=engine, prediction=prediction)

    @staticmethod
    def makeId(gameId, playerId):
        return ((gameId + '/') + playerId)

class BasicGameActivationBSONHandler():

    @staticmethod
    def reads(bson):
        return BasicGameActivation(id=bson['_id'], gameId=bson['gameId'], playerId=bson['userId'], engine=bson['engine'], prediction=bson['predic

In [119]:
[t[2] for t in unique]

['List[BasicGameActivation]\n',
 'GameID\n',
 'BasicGameActivation\n',
 'List[BasicGameActivation]\n',
 'Opt[int]\n',
 'PlayerID\n',
 'PlayerID\n',
 'PlayerID\n',
 'GameID\n',
 'BasicGameActivation\n',
 'Dict\n',
 'bool\n',
 'Dict\n',
 'Prediction\n',
 'Prediction\n',
 'BasicGameActivation\n',
 'bool\n',
 'List[BasicGameActivation]\n',
 'BasicGameActivationID\n']

In [106]:
print(node)

In [107]:
print(value)

GameID



In [109]:
node.lineno

20

In [110]:
node.col_offset

31

In [96]:
print(transformed)


'Type used for pivot coll for basic game model training'
from default_imports import *
from modules.game.Game import GameID, PlayerID
from pymongo.collection import Collection
BasicGameActivationID = NewType('BasicGameActivationID', str)
Prediction = NewType('Prediction', int)

class BasicGameActivation(NamedTuple('BasicGameActivation', [('id', BasicGameActivationID), ('gameId', GameID), ('playerId', PlayerID), ('engine', bool), ('prediction', int)])):

    @staticmethod
    def fromPrediction(gameId, playerId, prediction, engine):
        return BasicGameActivation(id=((gameId + '/') + playerId), gameId=gameId, playerId=playerId, engine=engine, prediction=prediction)

    @staticmethod
    def makeId(gameId, playerId):
        return ((gameId + '/') + playerId)

class BasicGameActivationBSONHandler():

    @staticmethod
    def reads(bson):
        return BasicGameActivation(id=bson['_id'], gameId=bson['gameId'], playerId=bson['userId'], engine=bson['engine'], prediction=bson['predic

In [72]:
value

'bool\n'

In [73]:
node.lineno

59

In [74]:
node.col_offset

44

In [75]:
node.ctx

In [76]:
transformed_ast

In [78]:
print(transformed)


'Type used for pivot coll for basic game model training'
from default_imports import *
from modules.game.Game import GameID, PlayerID
from pymongo.collection import Collection
BasicGameActivationID = NewType('BasicGameActivationID', str)
Prediction = NewType('Prediction', int)

class BasicGameActivation(NamedTuple('BasicGameActivation', [('id', BasicGameActivationID), ('gameId', GameID), ('playerId', PlayerID), ('engine', bool), ('prediction', int)])):

    @staticmethod
    def fromPrediction(gameId, playerId: PlayerID, prediction, engine: bool):
        return BasicGameActivation(id=((gameId + '/') + playerId), gameId=gameId, playerId=playerId, engine=engine, prediction=prediction)

    @staticmethod
    def makeId(gameId: GameID, playerId):
        return ((gameId + '/') + playerId)

class BasicGameActivationBSONHandler():

    @staticmethod
    def reads(bson) -> BasicGameActivation:
        return BasicGameActivation(id=bson['_id'], gameId=bson['gameId'], playerId=bson['userId'],

In [45]:
transformer.guard_value

In [43]:
%load_ext autoreload
%autoreload 2

In [41]:
print(transformed)


'Type used for pivot coll for basic game model training'
from default_imports import *
from modules.game.Game import GameID, PlayerID
from pymongo.collection import Collection
BasicGameActivationID = NewType('BasicGameActivationID', str)
Prediction = NewType('Prediction', int)

class BasicGameActivation(NamedTuple('BasicGameActivation', [('id', BasicGameActivationID), ('gameId', GameID), ('playerId', PlayerID), ('engine', bool), ('prediction', int)])):

    @staticmethod
    def fromPrediction(gameId, playerId: PlayerID, prediction, engine: bool):
        return BasicGameActivation(id=((gameId + '/') + playerId), gameId=gameId, playerId=playerId, engine=engine, prediction=prediction)

    @staticmethod
    def makeId(gameId: GameID, playerId):
        return ((gameId + '/') + playerId)

class BasicGameActivationBSONHandler():

    @staticmethod
    def reads(bson) -> BasicGameActivation:
        return BasicGameActivation(id=bson['_id'], gameId=bson['gameId'], playerId=bson['userId'],